<a href="https://colab.research.google.com/github/TOga1220/pytorch_tutorial/blob/main/200409_Wine_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワークによるワインの分類

In [ ]:
import torch                           # tensorを扱えるようにする
from torch.autograd import Variable    # 自動微分の関数
import torch.nn as nn                  # ニューラルネットワーク（nn）のモジュール
import torch.nn.functional as F         # 様々な活性化関数
import torch.optim as optim           # 最適化のアルゴリズム
from torch.utils.data import DataLoader, TensorDataset   # ユーティリティのデータを扱うためのデータローダー、テンソルデータセット

In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split 

In [ ]:
import pandas as pd

In [ ]:
## ワインデータセットの読み込み
wine = load_wine()

In [ ]:
wine

{'DESCR': '.. _wine_dataset:\n\nWine recognition dataset\n------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 178 (50 in each of three classes)\n    :Number of Attributes: 13 numeric, predictive attributes and the class\n    :Attribute Information:\n \t\t- Alcohol\n \t\t- Malic acid\n \t\t- Ash\n\t\t- Alcalinity of ash  \n \t\t- Magnesium\n\t\t- Total phenols\n \t\t- Flavanoids\n \t\t- Nonflavanoid phenols\n \t\t- Proanthocyanins\n\t\t- Color intensity\n \t\t- Hue\n \t\t- OD280/OD315 of diluted wines\n \t\t- Proline\n\n    - class:\n            - class_0\n            - class_1\n            - class_2\n\t\t\n    :Summary Statistics:\n    \n    ============================= ==== ===== ======= =====\n                                   Min   Max   Mean     SD\n    ============================= ==== ===== ======= =====\n    Alcohol:                      11.0  14.8    13.0   0.8\n    Malic Acid:                   0.74  5.80    2.34  1.12\n    Ash:           

In [ ]:
wine_data = pd.DataFrame(wine.data,columns=wine.feature_names )
print(wine_data.shape)

(178, 13)


In [ ]:
# ラベルが0,1のものだけ使用
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [ ]:
wine_target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size = 0.25) 

In [ ]:
len(train_X)

97

In [ ]:
len(test_X)

33

## Pytorchへのテンソル変換

In [ ]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()   # integer
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()   # integer

In [ ]:
train_X

tensor([[1.3750e+01, 1.7300e+00, 2.4100e+00,  ..., 1.1500e+00, 2.9000e+00,
         1.3200e+03],
        [1.2210e+01, 1.1900e+00, 1.7500e+00,  ..., 1.2800e+00, 3.0700e+00,
         7.1800e+02],
        [1.1870e+01, 4.3100e+00, 2.3900e+00,  ..., 7.5000e-01, 3.6400e+00,
         3.8000e+02],
        ...,
        [1.2990e+01, 1.6700e+00, 2.6000e+00,  ..., 1.3100e+00, 3.5000e+00,
         9.8500e+02],
        [1.2000e+01, 9.2000e-01, 2.0000e+00,  ..., 1.3800e+00, 3.1200e+00,
         2.7800e+02],
        [1.1450e+01, 2.4000e+00, 2.4200e+00,  ..., 8.0000e-01, 3.3900e+00,
         6.2500e+02]])

In [ ]:
train = TensorDataset(train_X, train_Y) 

In [ ]:
train[0]

(tensor([1.3750e+01, 1.7300e+00, 2.4100e+00, 1.6000e+01, 8.9000e+01, 2.6000e+00,
         2.7600e+00, 2.9000e-01, 1.8100e+00, 5.6000e+00, 1.1500e+00, 2.9000e+00,
         1.3200e+03]), tensor(0))

In [ ]:
# ミニバッチ＝部分集合にわけてトレーニングを何回もする時に使用
train_loader = DataLoader(train, batch_size=15, shuffle=True)

## モデルの定義

In [ ]:
# PyTorchではtorch.nnモジュールでニューラルネットワークの基底クラスとなるModuleクラスを定義
# Netクラスはtorch.nn.Moduleクラスを継承する
class Net(nn.Module):
  # インスタンス生成時に呼ばれる初期化の命令
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(13, 4)     # 入力は13ノード　、中間層は128とする
    self.fc2 = nn.Linear(4, 2)      # 入力は128, 出力が2

    # 　__init__メソッドで定義した層が実際にどのようにつながっているか（ニューラルネットワークがどのように計算を連ねていくか）はforwardメソッドで定める。
  def forward(self, x):
    x = F.relu(self.fc1(x))   # 活性化関数relu
    x = self.fc2(x)
    return F.log_softmax(x, dim=0)


In [ ]:
model = Net()

## トレーニングの実行

In [ ]:
#　損失の計算
# 交差エントロピー
criterion = nn.CrossEntropyLoss()

In [ ]:
# 最適化の計算
# 最適化アルゴリズム　　　SGD：確率的勾配効果法
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(500):
  total_loss = 0    # 損失を格納するための変数を0で初期化

  for train_x, train_y in train_loader:    # train_loaderのミニバッチのデータの塊がそれぞれtrain_x,train_yにセットされる
    train_x, train_y = Variable(train_x), Variable(train_y)   # torch.autograd.Variable : テンソルをwrapして計算操作を記録
    optimizer.zero_grad()  # optimizerの勾配を最初は０に。学習する旅に増やす
    output = model(train_x)
    loss = criterion(output, train_y)
    loss.backward()    # 誤差を後方に伝播
    optimizer.step()  # で伝播したデータを使ってパラメータを更新
    total_loss += loss.data.item()

  if (epoch+1) % 60 == 0:
    print(epoch+1, total_loss)  

60 4.852030992507935
120 4.852030992507935
180 4.852030992507935
240 4.852030992507935
300 4.852030992507935
360 4.852030992507935
420 4.852030992507935
480 4.852030992507935


## 精度の計算

In [ ]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = torch.max(model(test_x).data, 1)[1]        # 出力が0or1なのか。
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

In [ ]:
accuracy

0.48484848484848486